In [ ]:
// Pastikan 'lembang2' sudah didefinisikan sebagai ee.FeatureCollection atau ee.Geometry
// Contoh:
// var lembang2 = ee.FeatureCollection('users/YOUR_USERNAME/YOUR_ASSET_NAME');
// Map.centerObject(lembang2, 10);
// Map.addLayer(lembang2, {}, 'Lembang Boundary');

// --- Fungsi Pembantu ---

// Cloud Masking Function for Landsat 8
var masking = function(img) {
  var cloudshadowBitmask = (1 << 4);
  var cloudshadowmask = (1 << 3);
  var qa = img.select('QA_PIXEL');
  var maskshadow = qa.bitwiseAnd(cloudshadowBitmask).eq(0);
  var maskcloud = qa.bitwiseAnd(cloudshadowmask).eq(0);
  var mask = maskshadow.and(maskcloud);
  return img.updateMask(mask)
            .copyProperties(img, ['system:time_start', 'system:index', 'CLOUD_COVER_LAND']);
};

// Scaling Factor Function for Landsat 8 Surface Reflectance Bands
var scaleNDVI = function applyScaleFactors(image) {
  // Hanya ambil band optik dan terapkan scaling
  var opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  return image.addBands(opticalBands, null, true);
};

// --- Definisi Dataset ---
var Landsat_8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2");

// --- Rentang Waktu dan Parameter ---
var startYear = 2013;
var startMonth = 5; // Mei 2013, karena data publik Landsat 8 dimulai sekitar 30 Mei 2013
var endYear = 2022;
var endMonth = 12; // Desember
var numSamples = 1000; // Jumlah titik sampel acak per bulan
var scaleForSampling = 30; // Resolusi asli Landsat untuk sampling (m)

// --- Fungsi untuk Memproses Data NDVI Bulanan ---
var processMonthlyNDVI = function(date) {
  date = ee.Date(date);
  var endDate = date.advance(1, 'month');

  // Filter Landsat 8 untuk bulan ini
  var L8_monthly_raw = Landsat_8.filterBounds(lembang2)
                                .filterDate(date, endDate);

  // Apply masking and scaling for optical bands
  var L8_monthly_processed = L8_monthly_raw
                            .map(masking)
                            .map(scaleNDVI); // Gunakan fungsi scaleNDVI

  // Hitung median (komposit) untuk bulan tersebut.
  // Pilih band yang relevan untuk NDVI: SR_B5 (NIR) dan SR_B4 (Red)
  var NDVI_Image_Raw = L8_monthly_processed.select(['SR_B5', 'SR_B4']).median().clip(lembang2);

  // **Penting:** Periksa apakah citra median memiliki band yang dibutuhkan
  var hasLandsatData = NDVI_Image_Raw.bandNames().contains('SR_B5') &&
                       NDVI_Image_Raw.bandNames().contains('SR_B4');

  if (!hasLandsatData) {
    // Debugging: uncomment baris di bawah untuk melihat bulan mana yang kosong
    // print('No valid NDVI data for ' + date.format('YYYY-MM'));
    return ee.FeatureCollection([]); // Kembalikan koleksi kosong jika tidak ada data
  }

  // Hitung NDVI
  var NDVI = NDVI_Image_Raw.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI');

  // --- Sampling Acak ---
  var samplePoints = NDVI.sample({
    region: lembang2,
    scale: scaleForSampling,
    numPixels: numSamples,
    geometries: true,
    seed: 0,
    tileScale: 16
  });

  // Tambahkan properti waktu ke setiap fitur sampel
  samplePoints = samplePoints.map(function(feature) {
    return feature.set({
      'year': date.get('year'),
      'month': date.get('month'),
      'date': date.format('YYYY-MM-dd')
    });
  });

  // Debugging: uncomment baris di bawah untuk melihat jumlah sampel per bulan
  // print('Samples for ' + date.format('YYYY-MM') + ':', samplePoints.size());

  return samplePoints;
};

// --- Buat daftar semua bulan dalam rentang waktu ---
var dates = [];
for (var y = startYear; y <= endYear; y++) {
  var currentMonth = (y === startYear) ? startMonth : 1;
  var lastMonth = (y === endYear) ? endMonth : 12;

  for (var m = currentMonth; m <= lastMonth; m++) {
    dates.push(ee.Date.fromYMD(y, m, 1));
  }
}

// Proses data untuk setiap bulan
var all_sampled_NDVI_data = ee.FeatureCollection(ee.List(dates).map(processMonthlyNDVI)).flatten();

// --- Ekspor Data ke Google Drive ---
Export.table.toDrive({
  collection: all_sampled_NDVI_data,
  description: 'NDVI_Monthly_Lembang_Samples_From_L8_Start',
  folder: 'GEE_Kekeringan_Data',
  fileFormat: 'CSV'
});

// --- Opsional: Visualisasi satu contoh bulan (Misal, Juli 2013) ---
/*
var testDate = ee.Date.fromYMD(2013, 7, 1); // Coba bulan Juli 2013
var L8_monthly_test_raw = Landsat_8.filterBounds(lembang2).filterDate(testDate, testDate.advance(1, 'month'));
var L8_monthly_test_processed = L8_monthly_test_raw.map(masking).map(scaleNDVI);
var NDVI_Image_Test_Raw = L8_monthly_test_processed.select(['SR_B5', 'SR_B4']).median().clip(lembang2);

// Pastikan ada band sebelum menghitung NDVI
var hasTestLandsatData = NDVI_Image_Test_Raw.bandNames().contains('SR_B5') &&
                         NDVI_Image_Test_Raw.bandNames().contains('SR_B4');

if (hasTestLandsatData) {
  var NDVI_test_image = NDVI_Image_Test_Raw.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI');
  var NDVIParams = {min: 0.1, max: 0.8, palette: ['brown', 'orange', 'yellow', 'green', 'darkgreen']};
  Map.addLayer(NDVI_test_image, NDVIParams, 'NDVI Test Image ' + testDate.format('YYYY-MM'));
} else {
  print('No valid data for NDVI test image for ' + testDate.format('YYYY-MM'));
}

Map.addLayer(lembang2, {}, 'Lembang Boundary');
Map.centerObject(lembang2, 10);
*/

// Print total sampel yang dihasilkan di konsol
print('Total samples for all months (NDVI only):', all_sampled_NDVI_data.size());